# UPS MV Framework

### Import Libraries

In [1]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.reporting import reports as rp, plots, formatter as fmt
from ipywidgets import interact, interact_manual, interactive
import numpy as np

PLAN = 'IBT'

### Model liability of Plan

In [2]:
liab_model = summary.get_liab_model(PLAN)

IndexError: index 0 is out of bounds for axis 0 with size 0

### Compute plan inputs

In [ ]:
pp_inputs = summary.get_pp_inputs(liab_model,PLAN)

### Initialize plan object and display relevant variables

In [ ]:
plan = summary.get_plan_params(pp_inputs)

pp_dict = plan.get_pp_dict()
@interact
def display_pp_inputs(variable=pp_dict.keys()):
    if variable == 'Corr' or variable == 'Cov':
        return plots.draw_heatmap(pp_dict[variable], half=False)
    else:
        return fmt.get_plan_styler(pp_dict[variable])

### Define bounds
* Bounds data is stored in **'data/plan_inputs/bounds.xlsx'** by plan

In [ ]:
bnds = dm.get_bounds(liab_model,plan=PLAN)

@interact
def view_update_bnds(asset = plan.bnds_dict['asset_list'],reset=False,
                     lower=plan.bnds_dict['lower_bnd'],upper=plan.bnds_dict['upper_bnd'],reset_asset=False):
    if lower:
        fmt.update_lower_bnds(bnds, asset,lower,plan)
    if upper:
        fmt.update_upper_bnds(bnds, asset,upper,plan)
    if reset:
        fmt.reset_bnds(bnds,PLAN)
    if reset_asset:
        fmt.reset_asset_bnds(bnds,asset,PLAN)
    return fmt.get_plan_styler(bnds)

### Define contraints to optimize for min and max return
* Current constraints:
    * 45% <= sum(Fixed Income Assets weights) <= 55%
    * sum(All plan assets weights (excluding Futures & Hedges)) = Funded Status Diff
    * Hedges weights <= sum(50% of Equity & Private Equity weights)
    * 15+ STRIPS weight >= sum(50% of Futures and 25% of Hedges weights)

In [ ]:
cons = (
        # 45% <= sum of Fixed Income Assets <= 55%
        {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.45*plan.funded_status},
        {'type': 'ineq', 'fun': lambda x: .55*plan.funded_status - np.sum(x[1:3])},
        #sum of all plan assets (excluding Futures and Hedges) = Funded Status Difference    
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1]) - x[3] + (1-plan.funded_status)},
        # Hedges <= 50% of Equity & PE
        {'type': 'ineq', 'fun': lambda x: (x[4]+x[6])*.5 - x[len(plan)-1]},
        # 15+ STRIPS >= sum(50% of Futures and 25% of Hedges weights)
        {'type': 'ineq', 'fun': lambda x: x[1] - (x[3]/2+x[len(plan)-1]/4)}
)

### Compute and Display MV Efficient Frontier portfolios

In [ ]:
#Get data for MV efficient frontier portfolios
plan.compute_eff_frontier(bnds,cons,num_ports=100)
fmt.get_port_styler(plan.ports_df)

### Display MV Asset Allocation

In [ ]:
#Asset Allocation Plot
aa_fig = plots.get_aa_fig(plan.ports_df)
aa_fig.show()

### Display MV Efficient Frontier

In [ ]:
#Plotly version of the Efficient Frontier plot
ef_fig = plots.get_ef_fig(plan.ports_df)
ef_fig.show()

### Export data to excel

In [ ]:
#Export Efficient Frontier portfoio data to excel
rp.get_ef_portfolios_report(PLAN+'_ef_portfolios', plan,bnds)